In [1]:
import re, datetime, pytz, subprocess, os, json
from botocore.exceptions import ClientError
from time import gmtime, strftime
import boto3
import redis
import mysql.connector
import pandas as pd
from farmhash import FarmHash32 as fhash
from boto3.dynamodb.conditions import Key, Attr

import logging
# parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(parent_dir)
# from logging_utils import logging_to_cloudwatch as ltc
from logging_utils import logging_to_cloudwatch as ltc
    
logger = ltc.log('/aws/preprocessing/second','from_dynamodb_to_rds')



# 접속정보 json파일 로드
with open("./.KEYS/DATA_SRC_INFO.json", "r") as f:
# with open("./DATA_SRC_INFO.json", "r") as f:
    access_info = json.load(f)
# key 정보 json파일 로드
with open("./.KEYS/SECOND_PREPROCESSING_KEY.json", "r") as f:
# with open("./API_KEYS.json", "r") as f:
    key = json.load(f)
# rds 접속 정보 json파일 로드
with open("./.KEYS/RDS.json", "r") as f:
# with open("./DATA_SRC_INFO.json", "r") as f:
    rds_info = json.load(f)


In [2]:
def get_ids_from_redis():
    """
    Retrieve all ID values stored in Redis.

    This function connects to a Redis database using the connection information
    provided in the `access_info` dictionary. It fetches all keys from the 
    Redis hash named 'compare_id' and decodes them from bytes to UTF-8 strings.

    Returns:
        list: A list of ID values (as strings) retrieved from Redis.
    """
    
    redis_ip = access_info['redis_conn_info']['ip']
    redis_port = access_info['redis_conn_info']['port']
    try:
        redis_client = redis.StrictRedis(host=redis_ip, port=redis_port, db=0)
        # redis에 적재되어있는 id 값 가져오기
        return [x.decode('utf-8') for x in redis_client.hkeys('compare_id')]
    except redis.exceptions.RedisError as e:
        logger.error(f"Redis connection error: {e}")
        return []


def get_data_from_dynamodb(ids):
    """
    Retrieve all data from the DynamoDB table 'precessed-data-table' that matches the provided IDs.

    This function connects to a DynamoDB database using the AWS credentials
    provided in the `key` dictionary. It queries the 'precessed-data-table' table
    to fetch all items that match the IDs provided in the `ids` list.

    The function handles pagination to ensure that all items are retrieved,
    even if they span multiple pages.

    Args:
        ids (list): A list of IDs to query in the DynamoDB table.

    Returns:
        list: A list of data items (as dictionaries) retrieved from the DynamoDB table that match the provided IDs.

    Raises:
        ClientError: If there is an error while accessing DynamoDB.
        Exception: For any unexpected errors that may occur during execution.
    """
    
    dynamodb = boto3.resource(
            'dynamodb',
            aws_access_key_id=key['aws_access_key_id'],
            aws_secret_access_key=key['aws_secret_key'],
            region_name=key['region']
        )    
    table = dynamodb.Table('precessed-data-table')

    # 빈 리스트에 추가적으로 받아오기
    data = []
    for id in ids:
        try:
            response = table.query(KeyConditionExpression=Key('pid').eq(id))
            data.append(response['Items'])  # 데이터 값을 리스트에 추가
        except ClientError as e:
            logger.error(f"[get_data_from_dynamodb()]ClientError: {e.response['Error']['Message']}")
#             print(f"ClientError: {e.response['Error']['Message']}")
            continue  # 다음 ID로 이동
        except Exception as e:
            logger.error(f"[get_data_from_dynamodb()]Unexpected error: (id: {id}){str(e)}")
#             print(f"Unexpected error: (id: {id}){str(e)}")
            continue  # 다음 ID로 이동

    df = pd.DataFrame(data)
    return df


def get_pid_from_rds():
    """
    Retrieve all PID values from the RDS database.

    This function connects to the RDS database using the connection information
    provided in the `rds_info` dictionary. It executes a SQL query to fetch all
    PID values from the 'Job_Info' and returns them as a list.

    Returns:
        list: A list of PID values (as strings) retrieved from the RDS database.

    Raises:
        mysql.connector.Error: If there is an error while connecting to the RDS
        or executing the SQL query.
    """
    # RDS 접속 정보
    rds_host = rds_info['host']
    username = rds_info['username']
    password = rds_info['password']
    database = rds_info['database']

    # RDS에 연결
    connection = mysql.connector.connect(
        host=rds_host,
        user=username,
        password=password,
        database=database
    )

    try:
        cursor = connection.cursor()
        # pid를 추출하는 쿼리
        sql = "SELECT pid FROM Job_Info"
        cursor.execute(sql)
        pids = cursor.fetchall()  # 모든 pid를 가져옴
        return [pid[0] for pid in pids]  # pid 리스트 반환

    except mysql.connector.Error as e:
        logger.error(f"[get_pid_from_rds()]Database error: {str(e)}")  # 에러 로그 출력
        return []  # 빈 리스트 반환

    finally:
        cursor.close()  # 커서 종료
        connection.close()  # 연결 종료


def compare_id(redis_list, rds_list):
    """
    Generate a list of IDs that are present in the DynamoDB list but not in the Redis list.

    This function takes two lists: one containing IDs from Redis and another containing IDs from DynamoDB.
    It removes duplicates from the DynamoDB list and returns a list of IDs that are not found in the Redis list.

    Args:
        r_list (list): A list of IDs retrieved from Redis.
        d_list (list): A list of IDs retrieved from DynamoDB.

    Returns:
        list: A list of unique IDs from the DynamoDB list that are not present in the Redis list.
    """
    rds_id_df = pd.DataFrame({"id":rds_list})
    rds_id_df.drop_duplicates(ignore_index=True, inplace=True)
    mask = rds_id_df.isin(redis_list)
    return list(rds_id_df[~mask].dropna().reset_index(drop=True).id)


def load_to_rds(df: pd.DataFrame):
    
    # RDS 접속 정보
    rds_host = rds_info['host']
    username = rds_info['username']
    password = rds_info['password']
    database = rds_info['database']

    # RDS에 연결
    connection = mysql.connector.connect(
        host=rds_host,
        user=username,
        password=password,
        database=database
    )
    cursor = connection.cursor()
    
    sql_query = """
        INSERT INTO job_information (
            pid, job_title, site_symbol, job_prefer, crawl_url, start_date, end_date, post_status,
            get_date, required_career, resume_required, crawl_domain, company_name, cid, job_requirements
        ) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            pid = VALUES(pid),
            job_title = VALUES(job_title),
            site_symbol = VALUES(site_symbol),
            job_prefer = VALUES(job_prefer),
            crawl_url = VALUES(crawl_url),
            start_date = VALUES(start_date),
            end_date = VALUES(end_date),
            post_status = VALUES(post_status),
            get_date = VALUES(get_date),
            required_career = VALUES(required_career),
            resume_required = VALUES(resume_required),
            crawl_domain = VALUES(crawl_domain),
            cid = VALUES(cid),
            company_name = VALUES(company_name),
            job_requirements = VALUES(job_requirements)
    """

    for i, values in df.iterrows():
        data = (
            values['pid'],
            values['job_title'],
            values['site_symbol'],
            f"[{', '.join(f"'{item}'" for item in values['job_prefer'])}]"  if isinstance(values['job_prefer'], list) else values['job_prefer'],
            values['crawl_url'],
            values['start_date'],
            values['end_date'],
            values['post_status'],
            values['get_date'],
            values['required_career'],
            values['resume_required'],
            values['crawl_domain'],
            values['cid'],
            values['company_name'],
            f"[{', '.join(f"'{item}'" for item in values['job_requirements'])}]" if isinstance(values['job_requirements'], list) else values['job_requirements']
        )
        try:
            cursor.execute(sql_query, data)
            connection.commit()
            logger.info(f"Data successfully load to rds")
            
        except mysql.connector.Error as error:
            logger.error(f"Error executing SQL query for PID {values['pid']}: {error}")  # 에러 로깅
        except Exception as e:
            logger.error(f"Unexpected error for PID {values['pid']}: {str(e)}")  # 예기치 않은 에러 로깅
    cursor.close()  # 커서 종료
    connection.close()  # 연결 종료

In [3]:
def find_final_id_list():
    logger.info(f"start getting ids from redis")
    #user function
    id_list_from_redis = get_ids_from_redis()

    # 추가된 id값 list 생성
    # rds id 값 가져와서 비교
    logger.info(f"start getting ids from rds")
    #user function
    id_list_from_rds = get_pid_from_rds()
    list_to_update = compare_id(id_list_from_redis, id_list_from_rds)
    list_to_update = set(id_list_from_redis) - set(id_list_from_rds)
    dynamodb = boto3.resource(
            'dynamodb',
            aws_access_key_id=key['aws_access_key_id'],
            aws_secret_access_key=key['aws_secret_key'],
            region_name=key['region']
        )    
    table = dynamodb.Table('precessed-data-table')
    tmp = table.scan(Select='ALL_ATTRIBUTES')
    precesed_data= list(map(lambda x: str(x['pid']), tmp['Items']))
    final_id = list(set(precesed_data).intersection(list_to_update))
    return final_id

In [4]:
final_id = find_final_id_list()

## DynamoDB ->  


In [8]:
from decimal import Decimal
def get_processed_data(final_id):
    final_id = list(map(Decimal, final_id))
    dynamodb = boto3.resource(
            'dynamodb',
            aws_access_key_id=key['aws_access_key_id'],
            aws_secret_access_key=key['aws_secret_key'],
            region_name=key['region']
        )    
    table = dynamodb.Table('precessed-data-table')
    data = []
    for id in final_id:
        try:
            response = table.query(KeyConditionExpression=Key('pid').eq(id))
            data.append(response['Items'][0])  # 데이터 값을 리스트에 추가
        except ClientError as e:
            print(f"[get_data_from_dynamodb()]ClientError: {e.response['Error']['Message']}")
    #            print(f"ClientError: {e.response['Error']['Message']}")
            continue  # 다음 ID로 이동
        except Exception as e:
            print(f"[get_data_from_dynamodb()]Unexpected error: (id: {id}){str(e)}")
    #            print(f"Unexpected error: (id: {id}){str(e)}")
            continue  # 다음 ID로 이동
    return data

In [9]:
result = pd.DataFrame(get_processed_data(final_id))

In [10]:
result

,site_symbol,indurstry_type,job_title,job_category,job_prefer,crawl_url,end_date,pid,post_status,required_career,crawl_domain,company_name,resume_required,start_date,get_date,dev_stack,cid,job_requirements
0,PRO,"[Technology, Software, Cloud]",Cloud 인프라 아키텍쳐 담당,"[Cloud Architect, Solution Architect, Infrastr...","[SCM / SRM 등 구매 관련 업무 경험, IT 및 AI 산업 시장 변화 동향에...",https://career.programmers.co.kr/api/job_posit...,,3171239853,True,True,https://career.programmers.co.kr/,엠로,True,2024-04-29,20240828,"[AWS EC2, Azure, Clean Architecture]",1850949135,"[학사 학위 취득, Azure/AWS 등 대형 Vendor Cloud 경험, Clo..."
1,WAN,"[Autonomous Driving, Robotics, AI, Technology]",Computer Vision Engineer Autonomous Driving,"[Computer Vision Engineer, Research Scientist,...","[자율 주행 및 로보틱스 관련 연구 개발 경험, GPS, IMU, 카메라, LIDA...",https://www.wanted.co.kr/wd/216430,,2042877461,,,www.wanted.co.kr,포티투닷(42dot),,,1725864041,"[C/C++, Python]",1846198512,"[유관 경력 5년차 이상, 박사 졸업 예정자 지원 가능, 컴퓨터 비전, 로보틱스, ..."
2,WAN,"[IT, Software, HR Tech]",Cloud 인사시스템 개발 및 운영 담당,"[Software Engineer, Backend Developer, Cloud E...","[솔루션 운영 및 유지보수 경험, Vue.js 경험]",https://www.wanted.co.kr/wd/240348,,906695789,,,www.wanted.co.kr,이수시스템,,,1725515580,"[Java, JSP, JQuery, Oracle DB, Spring framewor...",4067809027,"[관련 분야에서 5년 이상의 경력, 솔루션 기반의 운영 및 유지보수 경험, 팀워크 ..."
3,JK,"[logistics, delivery, food delivery, outsourci...","연3,150만/인센/정규전환 쿠팡이츠 가맹점 운영 관리 담당자 대규모 채용","[customer service, operations, management, adm...",[관련 경력 우대],https://www.jobkorea.co.kr/Recruit/GI_Read/455...,1728777600,3007989768,None,False,None,에이스휴먼파워㈜,True,1726185600,1726538937,[],555094931,"[쿠팡이츠 가맹점 운영 관리, 고객 응대, 데이터 관리, 문제 해결]"
4,JK,"[web development, web design, marketing, adver...",워드프레스 개발자 채용,"[web development, wordpress development]",[],https://www.jobkorea.co.kr/Recruit/GI_Read/455...,None,707219774,None,False,None,넛지커뮤니케이션즈,True,1726185600,1726234283,[],1652767634,[워드프레스 개발]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,JK,[Apparel],데님프로모션 완사입 (DENIM) 영업관리.생산관리 신입 경력직 채용,"[Sales, Production Management, Fashion, Textile]",[관련 경력자 우대],https://www.jobkorea.co.kr/Recruit/GI_Read/455...,None,1955750282,None,False,None,㈜인디고라인,True,1726358400,1726452536,[],438941241,"[데님 관련 경험, 영업 및 생산 관리 능력, 패션 트렌드에 대한 이해]"
1008,JK,"[telecommunications, marketing, retail]",준텔레콤 창원 유튜브 유튜브/숏폼 영상촬영 및 편집자 채용,"[video editing, content creation, digital mark...",[],https://www.jobkorea.co.kr/Recruit/GI_Read/455...,1728777600,1121659660,None,False,None,㈜준텔레콤,True,1726185600,1726234283,[],4034255832,"[유튜브, 숏폼 영상 촬영, 편집]"
1009,JK,"[telecommunications, retail]",KT본사직영 KT 플라자 매장 정규직 채용공고,"[sales, retail, customer service, telecommunic...",[],https://www.jobkorea.co.kr/Recruit/GI_Read/455...,1728777600,4185711601,None,False,None,㈜케이티엠앤에스,True,1726185600,1726234283,[],2232376917,"[KT 플라자 매장, 정규직]"
1010,JK,"[Staffing, Management Consulting, Business Con...",송파구/복지우수 글로벌 외국계 공구 기업 CS 채용,"[Customer Service, Technical Support, Call Cen...","[영어 능통자 우대, 관련 분야 경력자 우대]",https://www.jobkorea.co.kr/Recruit/GI_Read/455...,1728950400,1904068323,None,True,None,㈜맨파워코리아,True,1726358400,1726452536,[],3973390486,"[고객 상담 경험, 전화 상담 능력, 문제 해결 능력, 커뮤니케이션 능력]"


In [29]:
import numpy as np
def get_rds_pid_list():
    # RDS 접속 정보
    rds_host = rds_info['host']
    username = rds_info['username']
    password = rds_info['password']
    database = rds_info['database']
    # RDS에 연결
    connection = mysql.connector.connect(
        host=rds_host,
        user=username,
        password=password,
        database=database
    )
    cursor = connection.cursor()
    cursor.execute("SELECT pid FROM job_information")
    a1 = cursor.fetchall()
    rds_pid_list = list(map(lambda x : int(x[0]), list(np.array(a1))))
    cursor.close()
    connection.close()
    return rds_pid_list

In [30]:
def get_rds_did_list():
    # RDS 접속 정보
    rds_host = rds_info['host']
    username = rds_info['username']
    password = rds_info['password']
    database = rds_info['database']
    # RDS에 연결
    connection = mysql.connector.connect(
        host=rds_host,
        user=username,
        password=password,
        database=database
    )
    cursor = connection.cursor()
    cursor.execute("SELECT did FROM dev_stack")
    a2 = cursor.fetchall()
    rds_did_list = list(map(lambda x : str(x[0]), list(np.array(a2))))
    cursor.close()
    connection.close()
    return rds_did_list

In [26]:
get_rds_pid_list()

['1002166931',
 '1004194211',
 '1007487304',
 '1008770164',
 '1011538068',
 '1016517998',
 '1019982009',
 '1032975379',
 '1038371436',
 '1063838531',
 '1066487499',
 '1069235713',
 '1072360246',
 '1074245989',
 '107796729',
 '1078376823',
 '1078933166',
 '1094502403',
 '11033851',
 '1125012312',
 '115335239',
 '1156940915',
 '116863901',
 '1169406102',
 '1171952029',
 '1177062975',
 '117958826',
 '1181601622',
 '1185761351',
 '1186934202',
 '1187277159',
 '1192613966',
 '11926471',
 '1194200153',
 '119664908',
 '1196877658',
 '1208330643',
 '1216324633',
 '1239911436',
 '1243268763',
 '1248994446',
 '1254441829',
 '1254716025',
 '1259580082',
 '1260967132',
 '1271076498',
 '1284518176',
 '1287547261',
 '1295089143',
 '1299792269',
 '130033997',
 '1301926306',
 '1304596496',
 '1307116181',
 '1330064495',
 '1343586709',
 '1349004222',
 '1350729060',
 '1371076200',
 '1372828126',
 '1373153800',
 '1378927626',
 '1380415105',
 '1381429147',
 '1401323579',
 '1405285534',
 '1408637987',
 '141

In [13]:
def get_did_jobstack(pid):
    # RDS 접속 정보
    rds_host = rds_info['host']
    username = rds_info['username']
    password = rds_info['password']
    database = rds_info['database']
    # RDS에 연결
    connection = mysql.connector.connect(
        host=rds_host,
        user=username,
        password=password,
        database=database
    )
    cursor = connection.cursor()
    sql_query_tmp = f"""
        SELECT did FROM job_stack
        WHERE pid = {pid}
    """
    cursor.execute(sql_query_tmp)
    a3 = cursor.fetchall()
    rds_pid_did = list(map(lambda x : int(x[0]), list(np.array(a3))))
    
    cursor.close()
    connection.close()
    return rds_pid_did

In [14]:
def get_pid_jobstack():
    # RDS 접속 정보
    rds_host = rds_info['host']
    username = rds_info['username']
    password = rds_info['password']
    database = rds_info['database']
    # RDS에 연결
    connection = mysql.connector.connect(
        host=rds_host,
        user=username,
        password=password,
        database=database
    )
    cursor = connection.cursor()
    sql_query_tmp2 = """
    SELECT pid FROM job_stack
    """
    cursor.execute(sql_query_tmp2)
    a4 = cursor.fetchall()
    pid_list = list(map(lambda x : int(x[0]), list(np.array(a4))))
    cursor.close()
    connection.close()
    return pid_list

In [34]:
def insert_data(result):
    # RDS 접속 정보
    rds_host = rds_info['host']
    username = rds_info['username']
    password = rds_info['password']
    database = rds_info['database']
    # RDS에 연결
    connection = mysql.connector.connect(
        host=rds_host,
        user=username,
        password=password,
        database=database
    )
    cursor = connection.cursor()
    rds_pid_list= get_rds_pid_list()
    rds_did_list=get_rds_did_list()
    pid_list = get_pid_jobstack()
    
    sql_query = """
        INSERT INTO job_information (
            pid, job_title, site_symbol, job_prefer, crawl_url, start_date, end_date, post_status,
            get_date, required_career, resume_required, crawl_domain, company_name, cid, job_requirements
        ) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            pid = VALUES(pid),
            job_title = VALUES(job_title),
            site_symbol = VALUES(site_symbol),
            job_prefer = VALUES(job_prefer),
            crawl_url = VALUES(crawl_url),
            start_date = VALUES(start_date),
            end_date = VALUES(end_date),
            post_status = VALUES(post_status),
            get_date = VALUES(get_date),
            required_career = VALUES(required_career),
            resume_required = VALUES(resume_required),
            crawl_domain = VALUES(crawl_domain),
            cid = VALUES(cid),
            company_name = VALUES(company_name),
            job_requirements = VALUES(job_requirements)
    """
    sql_query2 = """
    INSERT INTO dev_stack (
        did, dev_stack
    ) 
    VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE
        dev_stack = VALUES(dev_stack)
    """
    
    sql_query3 = """
    INSERT INTO job_stack (
        pid, did
    ) 
    VALUES (%s, %s)
    """
    
    for i, values in result.iterrows():
        data = (
            values['pid'],
            values['job_title'],
            values['site_symbol'],
            f"[{', '.join(f"'{item}'" for item in values['job_prefer'])}]"  if isinstance(values['job_prefer'], list) else values['job_prefer'],
            values['crawl_url'],
            values['start_date'],
            values['end_date'],
            values['post_status'],
            values['get_date'],
            values['required_career'],
            values['resume_required'],
            values['crawl_domain'],
            values['cid'],
            values['company_name'],
            f"[{', '.join(f"'{item}'" for item in values['job_requirements'])}]" if isinstance(values['job_requirements'], list) else values['job_requirements']
        )
        try:
            if values['pid'] not in rds_pid_list:
                cursor.execute(sql_query, data)
                connection.commit()
                print(f"Data successfully load to rds")
            else:
                print(f"overload data to rds")
        except mysql.connector.Error as error:
            print(f"Error executing SQL query for PID {values['pid']}: {error}")  # 에러 로깅
        except Exception as e:
            print(f"Unexpected error for PID {values['pid']}: {str(e)}")  # 예기치 않은 에러 로깅
            
        for stack in values['dev_stack']:
            data2 = (str(fhash(stack)), stack)
            data3 = (values['pid'],int(fhash(stack)))
            try:
                if data2[0] not in rds_did_list:
                    cursor.execute(sql_query2, data2)
                    connection.commit()
                    print(f"Data successfully load to rds")
                else:
                    print("Data overload")
            except mysql.connector.Error as error:
                print(f"Error executing SQL query: for dev_stack {stack}: {error}")  # 에러 로깅
            except Exception as e:
                print(f"Unexpected error for dev_stack {stack}: {str(e)}")  # 예기치 않은 에러 로깅
            try:
                if int(data3[0]) in pid_list and data3[1] in get_did_jobstack(values['pid']):
                    print("Data overload")
                else:
                    cursor.execute(sql_query3, data3)
                    connection.commit()
                    print(f"Data successfully load to rds")
            except mysql.connector.Error as error:
                print(f"Error executing SQL query for PID {values['pid']}: {error}")  # 에러 로깅
            except Exception as e:
                print(f"Unexpected error for PID {values['pid']}: {str(e)}")  # 예기치 않은 에러 로깅
    cursor.close()  # 커서 종료
    connection.close()  # 연결 종료

In [35]:
insert_data(result)

overload data to rds
Data overload
Unexpected error for PID 3171239853: name 'pid_list' is not defined
Data overload
Unexpected error for PID 3171239853: name 'pid_list' is not defined
Data overload
Unexpected error for PID 3171239853: name 'pid_list' is not defined
overload data to rds
Data overload
Unexpected error for PID 2042877461: name 'pid_list' is not defined
Data overload
Unexpected error for PID 2042877461: name 'pid_list' is not defined
overload data to rds
Data overload
Unexpected error for PID 906695789: name 'pid_list' is not defined
Data overload
Unexpected error for PID 906695789: name 'pid_list' is not defined
Data overload
Unexpected error for PID 906695789: name 'pid_list' is not defined
Data overload
Unexpected error for PID 906695789: name 'pid_list' is not defined
Data overload
Unexpected error for PID 906695789: name 'pid_list' is not defined
Data overload
Unexpected error for PID 906695789: name 'pid_list' is not defined
Data overload
Unexpected error for PID 90

In [136]:
pid_list = get_pid_jobstack()

In [140]:
sql_query3 = """
    INSERT INTO job_stack (
        pid, did
    ) 
    VALUES (%s, %s)
"""
pid_list = get_pid_jobstack()
for i, values in result.iterrows():
    for stack in values['dev_stack']:
        data2 = (values['pid'],int(fhash(stack)))
        try:
            if int(data2[0]) in pid_list and data2[1] in get_crid_jobstack(values['pid']):
                print("Data overload")
            else:
                cursor.execute(sql_query3, data2)
                connection.commit()
                print(f"Data successfully load to rds")
        except mysql.connector.Error as error:
            print(f"Error executing SQL query for PID {values['pid']}: {error}")  # 에러 로깅
        except Exception as e:
            print(f"Unexpected error for PID {values['pid']}: {str(e)}")  # 예기치 않은 에러 로깅


Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data overload
Data o

In [29]:
# RDS 접속 정보
rds_host = rds_info['host']
username = rds_info['username']
password = rds_info['password']
database = rds_info['database']
# RDS에 연결
connection = mysql.connector.connect(
    host=rds_host,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()

sql_query_tmp = f"""
    SELECT crid FROM include_cartegory
"""
cursor.execute(sql_query_tmp)
a4 = cursor.fetchall()
rds_pid_did = list(map(lambda x : int(x[0]), list(np.array(a4))))


In [ ]:
# RDS 접속 정보
rds_host = rds_info['host']
username = rds_info['username']
password = rds_info['password']
database = rds_info['database']
# RDS에 연결
connection = mysql.connector.connect(
    host=rds_host,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()

sql_query_tmp = f"""
    SELECT crid FROM include_cartegory
    WHERE pid = {1173140312}
"""
cursor.execute(sql_query_tmp)
a = cursor.fetchall()


In [142]:
# RDS 접속 정보
rds_host = rds_info['host']
username = rds_info['username']
password = rds_info['password']
database = rds_info['database']
# RDS에 연결
connection = mysql.connector.connect(
    host=rds_host,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()

sql_query4 = """
    INSERT INTO cartegory (
        crid, job_cartegory
    ) 
    VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE
        job_cartegory = VALUES(job_cartegory)
"""
for i, values in result.iterrows():
    for cart in values['job_category']:
        data4 = (str(fhash(cart)), cart)
        try:
            cursor.execute(sql_query4, data4)
            connection.commit()
            print(f"Data successfully load to rds")
        except mysql.connector.Error as error:
            print(f"Error executing SQL query for PID {values['pid']}: {error}")  # 에러 로깅
        except Exception as e:
            print(f"Unexpected error for PID {values['pid']}: {str(e)}")  # 예기치 않은 에러 로깅
cursor.close()  # 커서 종료
connection.close()  # 연결 종료

Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data successfully load to rds
Data succe

In [ ]:
# RDS 접속 정보
rds_host = rds_info['host']
username = rds_info['username']
password = rds_info['password']
database = rds_info['database']
# RDS에 연결
connection = mysql.connector.connect(
    host=rds_host,
    user=username,
    password=password,
    database=database
)
cursor = connection.cursor()

sql_query4 = """
    INSERT INTO cartegory (
        crid, job_cartegory
    ) 
    VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE
        job_cartegory = VALUES(job_cartegory)
"""
sql_query5 = """
    INSERT INTO include_cartegory (
        crid, pid
    ) 
    VALUES (%s, %s)
"""
for i, values in result.iterrows():
    for cart in values['job_category']:
        data4 = (str(fhash(cart)), cart)
        data5 = (str(fhash(cart)),values['pid'])
        try:
            cursor.execute(sql_query4, data4)
            connection.commit()
            print(f"Data successfully load to rds")
            cursor.execute(sql_query5, data5)
            connection.commit()
            print(f"Data successfully load to rds")
        except mysql.connector.Error as error:
            print(f"Error executing SQL query for PID {values['pid']}: {error}")  # 에러 로깅
        except Exception as e:
            print(f"Unexpected error for PID {values['pid']}: {str(e)}")  # 예기치 않은 에러 로깅
cursor.close()  # 커서 종료
connection.close()  # 연결 종료

In [125]:
result

,site_symbol,indurstry_type,job_title,job_category,job_prefer,crawl_url,end_date,pid,post_status,required_career,crawl_domain,company_name,resume_required,start_date,get_date,dev_stack,cid,job_requirements
0,WAN,"[IT, Software, Technology]",Java 개발자,"[Java Developer, Software Engineer]","[관련 경력 1년 이상, SI, ERP, MES, POP 등 프로젝트 개발 유경험]",https://www.wanted.co.kr/wd/240609,,1919149588,,,www.wanted.co.kr,엠아이티,,,1725556052,[Java],3752537004,"[Java 개발에 대한 깊은 이해, 팀 협업 및 커뮤니케이션 능력]"
1,JK,"[Apparel, Fashion, Export]",의류 해외영업 관련 정규직 채용,"[International Sales, Export, Sales, Business ...",[null],https://www.jobkorea.co.kr/Recruit/GI_Read/455...,1727708400,1416842085,None,True,None,㈜기수케이트레이딩,True,1726066800,1726123094,[],3659021357,"[해외 영업, 의류 수출, 바이어 발굴, 수출 계약, 영문 서류 작성]"
2,WAN,"[IT, Software, AI, Manufacturing, Security]",울산 영상처리 Python 개발자,"[Python Developer, Backend Developer]","[최신 백엔드 기술 이해 및 습득 능력, REST API 디자인 직관, 유저 문제 ...",https://www.wanted.co.kr/wd/241663,1727308800,3486557072,None,None,www.wanted.co.kr,인터엑스,None,None,1726160852,"[Python, REST API, Fast API, Linux, Nginx, Apa...",1967293289,"[파이썬 백엔드 개발 경력 2년 이상, Nginx, Apache 웹서버 구축 및 최..."
3,PRO,"[Software, IoT]",c#.NET 기반 IoT 플랫폼 미들웨어 개발자,"[Software Engineer, Middleware Developer, IoT ...","[닷넷 기반 C 언어 미들웨어 개발 경험, 데이터베이스 지식 보유, 데이터 수집 장...",https://career.programmers.co.kr/api/job_posit...,None,3837504727,True,True,https://career.programmers.co.kr/,에스위너스,True,2024-09-12,1726125518,"[C#, .NET, TCP, UDP, RS232]",3294195024,"[3년 이상 경력, C#, .NET, 미들웨어 개발, 데이터베이스 지식, 데이터 수..."
4,PRO,"[AI, Robotics, Marine, Fintech]",Back-end Developer (Data Platform),"[Backend Developer, Data Engineer]","[Gunicorn/nginx/celery/AWS 사용 경험, Python 기반 RO...",https://career.programmers.co.kr/api/job_posit...,,808590014,True,True,https://career.programmers.co.kr/,씨드로닉스,True,2024-02-23,20240828,"[Python, Django, PostgreSQL, ROS, REST API, Gu...",3648260093,"[Django (Python) 기반 실무 개발 경력 2년 이상, Django RES..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,WAN,"[IT, Software, AI, Deep Learning]",Deep Learning 개발자,"[Deep Learning Developer, AI Developer]","[LLM AI Agent 개발 경험, Mobile Application 및 Web ...",https://www.wanted.co.kr/wd/241695,1727654400,2659269050,None,None,www.wanted.co.kr,비주얼신,None,None,1726160852,"[Python, Tensorflow, Keras, Pytorch, GPU, Deep...",777346143,"[새로운 연구/기술 Follow-up, 고속 병렬 처리 GPU 관련 개발 및 빅데이..."
368,PRO,"[Healthcare, Technology, Software]",백 엔드 개발자,"[Backend Developer, Software Engineer]","[제품, 서비스 공급 기업에서 서비스 런칭, 운영, 고도화 업무 수행 경험, 의료,...",https://career.programmers.co.kr/api/job_posit...,,1950952480,True,True,https://career.programmers.co.kr/,헬스피디아,True,2024-01-03,20240828,"[Python, Django, Java, AWS EC2, MySQL, Postgre...",2662912872,"[웹에 대한 전반적인 지식, 웹 통신의 흐름, 인프라에 대한 이해, 제품, 서비스 ..."
369,PRO,"[E-commerce, Supply Chain Management, Technology]","Senior, Backend engineer (SCM)","[Backend Developer, Software Engineer, Senior ...","[컴퓨터 과학 분야 석사학위 소지 혹은 5년 이상 직접적인 소프트웨어 개발 경험, ...",https://career.programmers.co.kr/api/job_posit...,,3721845878,True,True,https://career.programmers.co.kr/,Coupang,True,2024-06-26,20240828,"[Java, Spring, Kotlin, SQL, Spring Boot, AWS]",3572271363,"[컴퓨터 과학 전공 우대 혹은 그에 준하는 학사학위 소지, 최소 5년 이상 직접적인..."
370,PRO,"[Software, Technology, Operating System]","[TmaxOS] 동영상 플랫폼 개발, 개방형OS 및 SuperApp 개발","[Software Engineer, Web Developer, System Soft...","[Linux System, Server Architecture 이해, Web Ser...",https://career.programmers.co.kr/api/job_posit...,,100900925,True,False,https://career.programmers.co.kr/,티맥스티베로,True,2023-07-04,20240828,"[Linux, Web Service, REST API, Spring, NodeJS,...",4065120436,"[4년제 정규 대학 졸업자 및 졸업예정자 (학사 학위 이상), 해외여행 결격 사유 ..."
